In [6]:
from tensorflow.python.keras._impl.keras.preprocessing import sequence
from tensorflow.python.keras._impl.keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from tensorflow.python.keras._impl.keras.layers import Reshape, Flatten, Dropout, Concatenate, dot
from tensorflow.python.keras._impl.keras.optimizers import Adam
from tensorflow.python.keras._impl.keras.models import Model
from tensorflow.python.keras._impl.keras.layers import LSTM
from tensorflow.python import keras
from tensorflow.python.keras._impl.keras.layers.wrappers import TimeDistributed, Bidirectional

from tensorflow.python.keras._impl.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras._impl.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras._impl.keras.utils.data_utils import get_file

import tensorflow as tf
import tensorflow.contrib.eager as tfe
# tf.contrib.eager.enable_eager_execution()

import os
import numpy as np
import csv, json
from zipfile import ZipFile
from os.path import expanduser, exists
import pandas as pd

In [7]:
quora_path = expanduser('~/.kaggle/competitions/quora-question-pairs/')
quora_train = os.path.join(quora_path, 'train.csv')
quora_test = os.path.join(quora_path, 'test.csv')

In [16]:
# KERAS_DATASETS_DIRKERAS_D  = expanduser('~/.keras/datasets/')
# QUESTION_PAIRS_FILE_URL = 'http://qim.ec.quoracdn.net/quora_duplicate_questions.tsv'
# QUESTION_PAIRS_FILE = 'quora_duplicate_questions.tsv'
GLOVE_ZIP_FILE_URL = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'
GLOVE_ZIP_FILE = 'glove.840B.300d.zip'
# GLOVE_FILE = 'glove.840B.300d.txt'
GLOVE_FILE = './data/glove.6B/glove.6B.50d.txt'

Q1_TRAINING_DATA_FILE = 'q1_train.npy'
Q2_TRAINING_DATA_FILE = 'q2_train.npy'
LABEL_TRAINING_DATA_FILE = 'label_train.npy'
WORD_EMBEDDING_MATRIX_FILE = 'word_embedding_matrix.npy'
NB_WORDS_DATA_FILE = 'nb_words.json'
MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 25
EMBEDDING_DIM = 50

# Tokenizer 세팅

In [9]:
df_train = pd.read_csv(quora_train, encoding='utf-8')
df_train = df_train.dropna() #drop empty

In [11]:
questions = list(df_train.question1) + list(df_train.question2)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions)
word_index = tokenizer.word_index

question1_word_sequences = tokenizer.texts_to_sequences(df_train.question1)
question2_word_sequences = tokenizer.texts_to_sequences(df_train.question2)

print("Words in index: %d" % len(word_index))

Words in index: 95595


# Download and extract questions pair dataset

In [12]:
question1 = df_train['question1']
question2 = df_train['question2']
is_duplicate = df_train['is_duplicate']

with open(quora_train, encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
        question1.append(row['question1'])
        question2.append(row['question2'])
        is_duplicate.append(row['is_duplicate'])

print('Question pairs: %d' % len(question1))

KeyError: 'question1'

# Glove embedding 로드하기

In [14]:
# if not exists(quora_path  + GLOVE_ZIP_FILE):
#     zipfile = ZipFile(get_file(GLOVE_ZIP_FILE, GLOVE_ZIP_FILE_URL))
#     zipfile.extract(GLOVE_FILE, path=quora_path)
    
print("Processing", GLOVE_FILE)

embeddings_index = {}
with open(GLOVE_FILE, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

Processing ./data/glove.6B/glove.6B.50d.txt
Word embeddings: 400000


# Word embedding matrix 만들기

In [17]:
nb_words=min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

Null word embeddings: 35259


# 학습 데이터 만들기

In [18]:
q1_data = pad_sequences(question1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(question2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(is_duplicate, dtype=int)
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of question1 data tensor: (404287, 25)
Shape of question2 data tensor: (404287, 25)
Shape of label tensor: (404287,)


# 만든 데이터 저장하기

In [19]:
np.save(open(Q1_TRAINING_DATA_FILE, 'wb'), q1_data)
np.save(open(Q2_TRAINING_DATA_FILE, 'wb'), q2_data)
np.save(open(LABEL_TRAINING_DATA_FILE, 'wb'), labels)
np.save(open(WORD_EMBEDDING_MATRIX_FILE, 'wb'), word_embedding_matrix)
with open(NB_WORDS_DATA_FILE, 'w') as f:
    json.dump({'nb_words': nb_words}, f)